In [3]:
import yaml
import os
import pathlib


ROOT = pathlib.Path().resolve().parent

print("Set as base folder: ", ROOT)
os.chdir(ROOT)


Set as base folder:  /Users/andrey/PycharmProjects/DataMiningProjects


In [6]:
with open(ROOT / "config.yml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [27]:
#3
# Variables from config
API_KEY = config['api']['key']
API_VERSION = config['api']['api_version']
API_SERVICE_NAME = config['api']['api_service_name']

publishedAfter = config['api']['parameters']['date_after']
publishedBefore = config['api']['parameters']['date_before']

# Notebook specific variables

q = "маркс | марксизм"
search_type = "channel"
# Для поиска


In [28]:
# 4

import googleapiclient.discovery as api

In [29]:
# 5

youtube = api.build(
    API_SERVICE_NAME,
    API_VERSION,
    developerKey = API_KEY
)

In [30]:
# 6

import pandas as pd

request = youtube.search().list(
    part="snippet",
    q=q,
    type=search_type,
    maxResults = 50,
    regionCode="RU"
)

response = request.execute()

response

{'kind': 'youtube#searchListResponse',
 'etag': 'bEuJIAHLg8nzOOwLuE0DXWyZS9M',
 'nextPageToken': 'CDIQAA',
 'regionCode': 'US',
 'pageInfo': {'totalResults': 47618, 'resultsPerPage': 50},
 'items': [{'kind': 'youtube#searchResult',
   'etag': '9JTGAjeUb3Q0_-tIzmvTbUaD6BY',
   'id': {'kind': 'youtube#channel', 'channelId': 'UCBi2mrWuNuyYy4gbM6fU18Q'},
   'snippet': {'publishedAt': '2006-08-07T23:12:21Z',
    'channelId': 'UCBi2mrWuNuyYy4gbM6fU18Q',
    'title': 'ABC News',
    'description': 'ABC News is your daily news outlet for breaking national and world news, video news, exclusive interviews and 24/7 live ...',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/OOT0eq8LoW8V3PMURGYuk0kRSgRGP7EsuGZoODhHBcg1jCB1qkuIAxS-XGnjKX9Q6rf-WKTvkQ=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/OOT0eq8LoW8V3PMURGYuk0kRSgRGP7EsuGZoODhHBcg1jCB1qkuIAxS-XGnjKX9Q6rf-WKTvkQ=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/OOT0eq8LoW8

In [31]:
# 7
df_supplemented = pd.json_normalize(response['items'])

In [32]:
# 8
# Цикл для прохода по всем следующим страницам с выдачей:
i = 0
while 'nextPageToken' in response.keys():
    request = youtube.search().list(
            part ="snippet",
            q = q,
            type = search_type,
            maxResults=50,
            pageToken = response['nextPageToken'],
            regionCode="RU"

        )
    response = request.execute()

    print(f'Итерация БЕЗ включения аргумента order №{i}')
    i += 1

    df_additional = pd.json_normalize(response['items'])
    df_supplemented = pd.concat([df_supplemented, df_additional])



print(f"Искомых объектов {response['pageInfo']['totalResults']}", \
      f"а найденных БЕЗ включения каких-либо значений аргумента order {len(df_supplemented.drop_duplicates('id.channelId'))}")

Итерация БЕЗ включения аргумента order №0
Итерация БЕЗ включения аргумента order №1
Итерация БЕЗ включения аргумента order №2
Итерация БЕЗ включения аргумента order №3
Итерация БЕЗ включения аргумента order №4
Итерация БЕЗ включения аргумента order №5
Итерация БЕЗ включения аргумента order №6
Итерация БЕЗ включения аргумента order №7
Итерация БЕЗ включения аргумента order №8
Итерация БЕЗ включения аргумента order №9
Искомых объектов 47294 а найденных БЕЗ включения каких-либо значений аргумента order 449


In [33]:
# 9
df_supplemented = df_supplemented.drop_duplicates()

df_supplemented = df_supplemented.reset_index(drop=True)

print('Уникальных объектов', len(df_supplemented))

Уникальных объектов 456


In [36]:
# 9
df_supplemented.to_excel('data/news_search_result_inorder.xlsx')